In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install cleverhans==3.0.1 tensorflow==1.8.0

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 49.1MB 1.4MB/s 
     |████████████████████████████████| 51kB 23.4MB/s 
     |████████████████████████████████| 163kB 39.1MB/s 
     |████████████████████████████████| 3.1MB 40.4MB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107221 sha256=763705273af572306ffabb2781c41e837810f14fdb2e0221ca4fcaac37724fe9
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.8.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found exist

In [0]:
home_dir = '/content/gdrive/My Drive/Colab Notebooks/newlogic'
import sys
sys.path.append(home_dir)

In [0]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [0]:
from cleverhans.attacks import FastGradientMethod
from cleverhans.model import CallableModelWrapper
from cleverhans.utils import AccuracyReport
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf

In [0]:
import torch
from torchvision import datasets, transforms

In [0]:
import pandas as pd

In [0]:
BATCH_SIZE = 128

In [0]:
def evaluate_model(path, ord=np.inf, epsilon=0.3):
    print('Evaluating model ' + path)
    torch_model_orig = torch.load(path)
    torch_model = lambda x: torch_model_orig(x)[0] # to standard format
    with tf.Session() as sess:
        x_op = tf.placeholder(tf.float32, shape=(None, 1, 28, 28,))    
        tf_model_fn = convert_pytorch_model_to_tf(torch_model, out_dims=10)    
        cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')
        # Create an FGSM attack
        fgsm_op = FastGradientMethod(cleverhans_model, sess=sess)
        fgsm_params = {'ord': ord, 'eps': epsilon, 'clip_min': 0., 'clip_max': 1.}
        adv_x_op = fgsm_op.generate(x_op, **fgsm_params)
        adv_preds_op = tf_model_fn(adv_x_op)
        test_loader = torch.utils.data.DataLoader(
          datasets.MNIST(home_dir + '/mnist', train=False, transform=transforms.ToTensor()),
          batch_size=BATCH_SIZE)
        # Run an evaluation of our model against fgsm
        total = 0
        correct = 0
        for xs, ys in tqdm(test_loader):
            adv_preds = sess.run(adv_preds_op, feed_dict={x_op: xs})
            correct += (np.argmax(adv_preds, axis=1) == ys.cpu().detach().numpy()).sum()
            total += test_loader.batch_size
    acc = float(correct) / total
    print('Adv accuracy: {:.3f}'.format(acc * 100))
    return acc

In [0]:
models = pd.read_csv(home_dir + '/mnist-models.csv')

In [0]:
attack_specs = [
    {'ord': np.inf, 'epsilon': 0.1},
    {'ord': np.inf, 'epsilon': 0.2},
    {'ord': np.inf, 'epsilon': 0.3},
    {'ord': 2, 'epsilon': 1.5},
    {'ord': 1, 'epsilon': 12},
]

In [0]:
results = []
for attack_spec in attack_specs:
    for model_name in models.name:
        acc = evaluate_model(home_dir + '/output/' + model_name + '.pkl', **attack_spec)
        results.append({
            'model_name': model_name,
            'accuracy': acc,
            'attack_type': 'FGM',
            **attack_spec
        })

  0%|          | 0/79 [00:00<?, ?it/s]

Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relu.pkl


  0%|          | 0/79 [00:00<?, ?it/s]

Adv accuracy: 85.433
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relu-kernel.pkl


  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Adv accuracy: 96.964
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relu-maxfit_l1_01_05.pkl


  0%|          | 0/79 [00:00<?, ?it/s]

Adv accuracy: 95.807
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relu-maxfit_l2_01_05.pkl


100%|██████████| 79/79 [00:03<00:00, 20.64it/s]

In [0]:
pd.DataFrame(results).to_csv(home_dir + '/output/mnist-fgm-results.csv', index=False)

In [14]:
results = []
for attack_spec in attack_specs:
    for model_name in models.name.iloc[-1:]:
        evaluate_model(home_dir + '/output/' + model_name + '.pkl', **attack_spec)

Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_out-max_margin.pkl


  0%|          | 0/79 [00:00<?, ?it/s]

Adv accuracy: 93.602
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_out-max_margin.pkl


  0%|          | 0/79 [00:00<?, ?it/s]

Adv accuracy: 82.516
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_out-max_margin.pkl


...

Adv accuracy: 64.458
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_out-max_margin.pkl


  0%|          | 0/79 [00:00<?, ?it/s]

Adv accuracy: 88.726
Evaluating model /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_out-max_margin.pkl


100%|██████████| 79/79 [00:06<00:00, 13.31it/s]

Adv accuracy: 97.033
